In [1]:
from typing import Any, TypedDict
from pydantic import BaseModel

class BaseData(TypedDict):
    id: int
    name: str
    value: float

class ExtendedData(BaseData):
    description: str

class MoreExtendedData(ExtendedData):
    extra_info: Any

In [2]:
class BaseProcessor(BaseModel):
    name: str = "base"
    def process(self, data: BaseData) -> BaseData:
        # Example processing logic
        processed_value = data["value"] * 2
        return {
            "id": data["id"],
            "name": self.name,
            "value": processed_value,
        }
    
class ExtendedProcessor(BaseProcessor):
    def process(self, data: ExtendedData) -> ExtendedData: # type: ignore[override]
        # Have to suppress reportIncompatibleMethodOverride because of input type change
        processed_value = data["value"] * 2
        return {
            "id": data["id"],
            "name": self.name,
            "value": processed_value,
            "description": data["description"].upper(),
        }
    
class ProcessorMixin(BaseProcessor):
    name: str = "mixin"
    def process(self, data: BaseData) -> BaseData:
        print(f"Mixin is processing data with id: {data['id']}")
        return super().process(data)

In [12]:
MixinCtor = type('MyProcessor', (ExtendedProcessor, ProcessorMixin), {})
MixinCtor.model_rebuild()
processor = MixinCtor()
processor.name # Supposed to be "mixin" due to MRO

'base'

In [13]:
MixinCtor.mro()

[abc.MyProcessor,
 __main__.ExtendedProcessor,
 __main__.ProcessorMixin,
 __main__.BaseProcessor,
 pydantic.main.BaseModel,
 object]

In [5]:
class GenericProcessor[T: BaseData](BaseModel):
    name: str = "base"
    def process(self, data: T) -> BaseData:
        # Example processing logic
        processed_value = data["value"] * 2
        return {
            "id": data["id"],
            "name": self.name,
            "value": processed_value,
        }
    
class GenericExtendedProcessor[T: ExtendedData](GenericProcessor[T]):
    def process(self, data: T) -> ExtendedData:
        # No need to ignore override here
        processed_value = data["value"] * 2
        return {
            "id": data["id"],
            "name": self.name,
            "value": processed_value,
            "description": data["description"].upper(),
        }
    
class GenericProcessorMixin[T: BaseData](GenericProcessor[T]):
    name: str = "mixin"
    def process(self, data: T) -> BaseData:
        print(f"Generic Mixin is processing data with id: {data['id']}")
        return super().process(data)

In [10]:
GenericMixinCtor = type('MyGenericProcessor', (GenericExtendedProcessor, GenericProcessorMixin), {})
GenericMixinCtor.model_rebuild()
generic_processor = GenericMixinCtor()
generic_processor.name # Supposed to be "mixin" due to MRO

'base'

In [11]:
GenericMixinCtor.mro()

[abc.MyGenericProcessor,
 __main__.GenericExtendedProcessor,
 __main__.GenericProcessor[TypeVar],
 __main__.GenericProcessorMixin,
 __main__.GenericProcessor[TypeVar],
 __main__.GenericProcessor,
 pydantic.main.BaseModel,
 typing.Generic,
 object]

In [31]:
class Base(BaseModel):
    name: str = "base"

class ExtendedBase(Base):
    pass

class Mixin(Base):
    name: str = "mixin"

MixinCtor = type('MyProcessor', (ExtendedBase, Mixin), {})
MixinCtor.model_rebuild()
obj = MixinCtor()
obj.name, MixinCtor.mro()

('base',
 [abc.MyProcessor,
  __main__.ExtendedBase,
  __main__.Mixin,
  __main__.Base,
  pydantic.main.BaseModel,
  object])

In [32]:
ExtendedBase.model_fields

{'name': FieldInfo(annotation=str, required=False, default='base')}

In [30]:
class MixinClass(ExtendedBase, Mixin):
    pass

obj2 = MixinClass()
obj2.name, MixinClass.mro()

('base',
 [__main__.MixinClass,
  __main__.ExtendedBase,
  __main__.Mixin,
  __main__.Base,
  pydantic.main.BaseModel,
  object])

In [28]:
class Base():
    name: str = "base"

class ExtendedBase(Base):
    pass

class Mixin(Base):
    name: str = "mixin"

MixinCtor = type('MyProcessor', (ExtendedBase, Mixin), {})
processor = MixinCtor()
processor.name, MixinCtor.mro()

('mixin',
 [__main__.MyProcessor,
  __main__.ExtendedBase,
  __main__.Mixin,
  __main__.Base,
  object])